# Задача класифікації пацієнтів щодо схильності до серцевих нападів.

Предметною областю є медицина, а саме - кардіологія (серцеві напади). Нашою метою є класифікація пацієнтів за наданими в датасеті характеристиками для прогнозування, чи схильні вони до серцевого нападу.

Використовуємо датасет з відкритого ресурсу, Kaggle: https://www.kaggle.com/rashikrahmanpritom/heart-attack-analysis-prediction-dataset

Ознайомимось попередньо з даними файлу:
- Age: вік пацієнта
- Sex: стать пацієнта
- exang: стенокардія, спричинена фізичними вправами (1 = так; 0 = ні)
- ca: кількість основних суден (0-3)
- cp: Біль у грудях тип болю в грудях
- Value 1: типова стенокардія
- Value 2: атипова стенокардія
- Value 3: неангінозний біль
- Value 4: безсимптомно
- trtbps: артеріальний тиск у спокої (у мм рт. ст.)
- chol: холесторал у мг / дл, отриманий за допомогою датчика ІМТ
- fbs: (цукор у крові натще> 120 мг / дл) (1 = істинно; 0 = хибно)
- rest_ecg: результати електрокардіографічного спокою
- Value 0: нормальне
- Value 1: наявність аномалії хвилі ST-T (інверсія зубця T та / або підвищення ST або депресія> 0,05 мВ)
- Value 2: показ імовірної або певної гіпертрофії лівого шлуночка за критеріями Естеса
- thalach: досягнутий максимальний пульс
- target: 0 = менше шансів серцевого нападу 1 = більше шансів серцевого нападу

Для вирішення задачі використовуватимемо такі бібліотеки як pandas, numpy, sklearn, seaborn та інші.

Щодо ML-алгоритму, плануємо застосувати декілька, щоб порівняти результати та обрати найефективніший.

In [ ]:
import pandas as pd
import pandas_profiling as pp

import seaborn as sns
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
!pip install pywaffle
from pywaffle import Waffle
from sklearn import tree
import sklearn.metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import warnings
from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import ConfusionMatrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, precision_score, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
warnings.filterwarnings("ignore")

from IPython.display import Image

In [ ]:
import os

df = pd.read_csv(os.path.join('/kaggle/input/heart-attack-analysis-prediction-dataset/','heart.csv'))
df

In [ ]:
df.info()

In [ ]:
# видаляємо дублікати
df = df.drop_duplicates()

Попередньо побачили, що в датасеті немає рядків з пропущеними записами, тож функція FillNa не знадобиться.

In [ ]:
# застосування   описової   статистики
df.describe()

In [ ]:
# проведення  кореляційного  та  причинно-наслідкового  аналізів 
df.corr()

In [ ]:
sns.heatmap(df)

In [ ]:
sns.pairplot(df)

In [ ]:
# конструювання  ознак

df = df[df.chol<380]
df = df[df.trtbps<190]
df = df[df.thalachh>75]

In [ ]:
# розбиття   набору   даних   на   тренувальний та  тестовий  набори  даних

X = df.drop('output', axis = 1)
y = df.output
X = StandardScaler().fit(X).transform(X)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=10)

In [ ]:
np.info(X_train)

In [ ]:
np.info(X_test)

Для вирішення посталеної задачі будемо використовувати 5 типових методів методів:
- DecisionTreeClassifier(Це навчання є одним з підходів моделювання прогнозів, які використовуються в статистиці, інтелектуальному аналізі даних і навчанні машин. Моделі дерев, в яких цільова змінна може приймати дискретний набір значень, називаються деревами класифікації)
- GaussianNB(простий імовірнісний класифікатор, заснований на застосуванні теореми Байеса зі строгими (наївними) припущеннями про незалежність.)
- XGBClassifier(До важливих властивостей XGBoost, які відрізняють її від інших алгоритмів підсилювання градієнту, належать: Розумне штрафування дерев, Пропорційне скорочування листових вузлів, Ньютонове підсилювання, Додатковий параметр рандомізації, Втілення на окремих, розподілених системах, та позаядрових обчисленнях.)
- LogisticRegression(це статистична модель, використовувана для прогнозування ймовірності виникнення деякої події шляхом його порівняння з логістичної кривої. Ця регреcсія видає відповідь у вигляді ймовірності бінарного події (1 або 0).)
- RandomForestClassifier(ансамблевий метод машинного навчання для класифікації, регресії та інших завдань, які оперують за допомогою побудови численних дерев прийняття рішень під час тренування моделі і продукують моду для класів (класифікацій) або усереднений прогноз (регресія) побудованих дерев. Недоліком є схильність до перенавчання.)

Для порівняння ефективності роботи моделей будемо враховувати:
- матрицю помилок
- trainning та test accuracy(У багаторабельній класифікації ця функція обчислює точність підмножини: набір лейблів, передбачених для зразка, повинен точно відповідати відповідному набору лейблів у y_true.)
- precision(також називається позитивна прогностична цінність - це частка відповідних інстанцій серед витків екземплярів)
- recall(також називається чутливість - це частка відповідних випадків, які були отримані)
- f1(у статистичному аналізі бінарної класифікації F-оцінка або F-міра є кількістю точних тестів)

In [ ]:
Image(filename='/kaggle/input/images/form.jpg')

In [ ]:
Image(filename='/kaggle/input/images/from2.jpg')

Імплементація обраних методів моделювання

In [ ]:
# DecisionTreeClassifier

model = tree.DecisionTreeClassifier(max_depth=15)
clf = model.fit(X_train, y_train)
y_predict=clf.predict(X_train)
train_accuracy = sklearn.metrics.accuracy_score(y_predict, y_train)
y_valid=clf.predict(X_test)
y_dt=y_valid
test_accuracy = sklearn.metrics.accuracy_score(y_valid, y_test)

fig=plt.figure(figsize=(12,5))
fig = plt.subplot(121)
plt.title("Confusion metric")
sns.heatmap(confusion_matrix(y_test, y_valid), annot = True)
fig = plt.subplot(122)
classes = ["No Heart Attack","Heart Attack"]
visualizer = ClassificationReport(model, classes=classes, support=True)
visualizer.fit(X_train, y_train)       
visualizer.score(X_test, y_test)        
visualizer.show()  

print("Training accuracy",train_accuracy)
print("Test accuracy",test_accuracy)

In [ ]:
# GaussianNB

model = GaussianNB()
clf = model.fit(X_train, y_train)
y_predict=clf.predict(X_train)
train_accuracy = sklearn.metrics.accuracy_score(y_predict, y_train)
y_valid=clf.predict(X_test)
y_nb=y_valid
test_accuracy = sklearn.metrics.accuracy_score(y_valid, y_test)

fig=plt.figure(figsize=(12,5))
fig = plt.subplot(121)
plt.title("Confusion metric")
sns.heatmap(confusion_matrix(y_test, y_valid), annot = True)
fig = plt.subplot(122)
classes = ["No Heart Attack","Heart Attack"]
visualizer = ClassificationReport(model, classes=classes, support=True)
visualizer.fit(X_train, y_train)       
visualizer.score(X_test, y_test)        
visualizer.show()  

print("Training accuracy",train_accuracy)
print("Test accuracy",test_accuracy)

In [ ]:
# XGBClassifier

model = XGBClassifier()
clf = model.fit(X_train, y_train)
y_predict=clf.predict(X_train)
train_accuracy = sklearn.metrics.accuracy_score(y_predict, y_train)
y_valid=clf.predict(X_test)
y_xg=y_valid
test_accuracy = sklearn.metrics.accuracy_score(y_valid, y_test)

fig=plt.figure(figsize=(12,5))
fig = plt.subplot(121)
plt.title("Confusion metric")
sns.heatmap(confusion_matrix(y_test, y_valid), annot = True)
fig = plt.subplot(122)
classes = ["No Heart Attack","Heart Attack"]
visualizer = ClassificationReport(model, classes=classes, support=True)
visualizer.fit(X_train, y_train)       
visualizer.score(X_test, y_test)        
visualizer.show()  

print("Training accuracy",train_accuracy)
print("Test accuracy",test_accuracy)

In [ ]:
# LogisticRegression

model = LogisticRegression()
clf = model.fit(X_train, y_train)
y_predict=clf.predict(X_train)
train_accuracy = sklearn.metrics.accuracy_score(y_predict, y_train)
y_valid=clf.predict(X_test)
y_lr=y_valid
test_accuracy = sklearn.metrics.accuracy_score(y_valid, y_test)

fig=plt.figure(figsize=(12,5))
fig = plt.subplot(121)
plt.title("Confusion metric")
sns.heatmap(confusion_matrix(y_test, y_valid), annot = True)
fig = plt.subplot(122)
classes = ["No Heart Attack","Heart Attack"]
visualizer = ClassificationReport(model, classes=classes, support=True)
visualizer.fit(X_train, y_train)       
visualizer.score(X_test, y_test)        
visualizer.show()  

print("Training accuracy",train_accuracy)
print("Test accuracy",test_accuracy)

In [ ]:
# RandomForestClassifier

model = RandomForestClassifier(n_estimators=3000)
clf = model.fit(X_train, y_train)
y_predict=clf.predict(X_train)
train_accuracy = sklearn.metrics.accuracy_score(y_predict, y_train)
y_valid=clf.predict(X_test)
y_rf=y_valid
test_accuracy = sklearn.metrics.accuracy_score(y_valid, y_test)

fig=plt.figure(figsize=(12,5))
fig = plt.subplot(121)
plt.title("Confusion metric")
sns.heatmap(confusion_matrix(y_test, y_valid), annot = True)
fig = plt.subplot(122)
classes = ["No Heart Attack","Heart Attack"]
visualizer = ClassificationReport(model, classes=classes, support=True)
visualizer.fit(X_train, y_train)       
visualizer.score(X_test, y_test)        
visualizer.show()  

print("Training accuracy",train_accuracy)
print("Test accuracy",test_accuracy)

In [ ]:
lr_df = pd.DataFrame(data=[f1_score(y_test,y_lr),accuracy_score(y_test, y_lr), recall_score(y_test, y_lr), precision_score(y_test, y_lr), roc_auc_score(y_test, y_lr)], 
             columns=['Logistic Regression'], index=["F1","Accuracy", "Recall", "Precision", "ROC AUC Score"])
rf_df = pd.DataFrame(data=[f1_score(y_test,y_rf),accuracy_score(y_test, y_rf), recall_score(y_test, y_rf),precision_score(y_test, y_rf), roc_auc_score(y_test, y_rf)], 
             columns=['Random Forest Score'],index=["F1","Accuracy", "Recall", "Precision", "ROC AUC Score"])
nb_df = pd.DataFrame(data=[f1_score(y_test,y_nb),accuracy_score(y_test, y_nb), recall_score(y_test, y_nb), precision_score(y_test, y_nb), roc_auc_score(y_test, y_nb)], 
             columns=['Naive Bayes'], index=["F1","Accuracy", "Recall", "Precision", "ROC AUC Score"])

xg_df = pd.DataFrame(data=[f1_score(y_test,y_xg),accuracy_score(y_test, y_xg), recall_score(y_test, y_xg), precision_score(y_test, y_xg), roc_auc_score(y_test, y_xg)], 
             columns=['XG Boost'], index=["F1","Accuracy", "Recall", "Precision", "ROC AUC Score"])
dt_df = pd.DataFrame(data=[f1_score(y_test,y_dt),accuracy_score(y_test, y_dt), recall_score(y_test, y_dt), precision_score(y_test, y_dt), roc_auc_score(y_test,y_dt)], 
             columns=['Decision Tree'], index=["F1","Accuracy", "Recall", "Precision", "ROC AUC Score"])


df_models = round(pd.concat([lr_df,rf_df,nb_df,dt_df,xg_df], axis=1),3)
colors = ["bisque","ivory","sandybrown","steelblue","lightsalmon"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

background_color = "white"

fig = plt.figure(figsize=(18,26))
gs = fig.add_gridspec(4, 2)
gs.update(wspace=0.1, hspace=0.5)
ax0 = fig.add_subplot(gs[0, :])

sns.heatmap(df_models.T, cmap=colormap,annot=True,fmt=".1%",vmin=0,vmax=0.95, linewidths=2.5,cbar=False,ax=ax0,annot_kws={"fontsize":16})
fig.patch.set_facecolor(background_color)
ax0.set_facecolor(background_color) 

ax0.text(0,-0.5,'Model Comparison',fontsize=20,fontweight='bold',fontfamily='serif')
plt.show()

Бачимо з таблиці порівняння моделей, що найкраще себе показали Логістична регресія та Випадковий ліс, але, в цілому, всі моделі дають гарний результат